In [1]:
import math
import json
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align import AlignInfo
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


Bad key "text.kerning_factor" on line 4 in
/Users/katekistler/anaconda3/envs/nextstrain/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
#divergence is calculated from consensus sequence at first time point

In [2]:
def separate_clades(cov, gene):
    if cov == 'hku1':
        clade_file = '../'+str(cov)+'/results/clades_full.json'
    else:
        if gene =='spike' or gene == 's1' or gene == 's2':
            clade_file = '../'+str(cov)+'/results/clades_spike.json'
        else:
            clade_file = '../'+str(cov)+'/results/clades_full.json'
    
    clade_lists = []
    with open(clade_file, "r") as clade_handle:
        clades = json.load(clade_handle)
        for node, v in clades['nodes'].items():
            if 'NODE' not in node:
                clade_lists.append({'clade':v['clade_membership'],
                                   'strain':node})

    clade_df = pd.DataFrame(clade_lists)
    return clade_df

In [3]:
def syn_nonsyn_histogram(cov, gene, clade):
    #Find frequency of nonsyn and syn SNPs
    input_file_root = '../'+str(cov)+'/auspice/seasonal_corona_'+str(cov)+'_'+str(gene)+'_root-sequence.json'
    input_file_alignment = '../'+str(cov)+'/results/aligned_'+str(cov)+'_'+str(gene)+'.fasta'
    metafile = '../'+str(cov)+'/results/metadata_'+str(cov)+'_'+str(gene)+'.tsv'
    

    meta = pd.read_csv(metafile, sep = '\t')
    meta.drop(meta[meta['date']=='?'].index, inplace=True)
    meta.dropna(subset=['date'], inplace=True)
    meta['year'] = meta['date'].str[:4].astype('int')
    
    
    if clade!= None:
        clade_df = separate_clades(cov, gene)
        meta = meta.merge(clade_df, on='strain')
        meta.drop(meta[meta['clade']!=clade].index, inplace=True)
    
    strains = meta['strain'].tolist()
                
    fraction_subs = []
    fraction_syn = []
    fraction_nonsyn = []

    #calculate divergence at all subsequent time points
    with open(input_file_alignment, "r") as aligned_handle:
        for virus in SeqIO.parse(aligned_handle, "fasta"):         
            if virus.id in strains:    

                #check
                if len(virus.seq) != len(root_seq):
                    print(virus)
                elif len(virus.seq) == len(root_seq):
                    count_total_unambiguous = 0
                    count_subs = 0
                    count_syn_subs = 0
                    count_nonsyn_subs = 0
                    for pos in range(len(root_seq)):
                        root_nt = str(root_seq[pos])
                        virus_nt = str(virus.seq[pos])
                        #skip ambiguous sites
                        if virus_nt != 'N':
                            if root_nt != 'N':
                                count_total_unambiguous+=1
                                if virus_nt != root_nt:
                                    count_subs+=1
                                    #determine syn or nonsyn
                                    codon = math.floor(pos/3)
                                    codon_pos = pos-(codon*3)
                                    if codon_pos == 0:
                                        codon_nt = virus.seq[pos:(pos+3)]
                                    elif codon_pos == 1:
                                        codon_nt = virus.seq[(pos-1):(pos+2)]
                                    elif codon_pos == 2:
                                        codon_nt = virus.seq[(pos-2):(pos+1)]
                                    codon_aa = codon_nt.translate()
                                    root_aa = root_aa_seq[codon]
                                    #skip ambiguous
                                    if root_aa != 'X':
                                        if codon_aa != root_aa:
                                            count_nonsyn_subs+=1
                                            nt_mut = str(root_nt)+str(pos)+str(virus_nt)+'nonsyn'
                                        elif codon_aa == root_aa:
                                            count_syn_subs+=1
                                            nt_mut = str(root_nt)+str(pos)+str(virus_nt)+'syn'
                                        if nt_mut not in nt_muts:
                                            nt_muts.append(nt_mut)
                    fraction_sub_virus = count_subs/count_total_unambiguous
                    fraction_subs.append(fraction_sub_virus)
                    fraction_syn_virus = count_syn_subs/count_total_unambiguous
                    fraction_syn.append(fraction_syn_virus)
                    fraction_nonsyn_virus = count_nonsyn_subs/count_total_unambiguous
                    fraction_nonsyn.append(fraction_nonsyn_virus)
    
    

In [4]:
syn_nonsyn_histogram('oc43', 'spike', 'A')

oc43/KF572819/039A_07/2007
oc43/KF572861/5617_07/2007
oc43/KF572842/4795_07/2007
oc43/KF572844/5240_07/2007
oc43/KF572847/5352_07/2007
oc43/KF572851/5445_07/2007
oc43/KF572860/5595_07/2007
oc43/KF572863/5656_07/2007
oc43/KF572864/69A_07/2007
oc43/KF572846/5345_07/2007
oc43/KF572850/5442_07/2007
oc43/KF572852/5472_07/2007
oc43/KF572855/5485_07/2007
oc43/KF572857/5517_07/2007
oc43/KF572854/5484_07/2007
oc43/KF572848/5370_07/2007
oc43/KF572853/5479_07/2007
oc43/LC506765/HCoV_OC43/Yamagata.JPN/11.2009_0765/2009
oc43/KF572843/4954_07/2007
oc43/KF572856/5508_07/2007
oc43/KF572868/892A_08/2008
oc43/KF572862/5625_07/2007
oc43/KF572845/5331_07/2007
oc43/KF572859/5566_07/2007
oc43/KF572820/079A_07/2007
oc43/KF572858/5519_07/2007
oc43/KF572821/10108_10/2010
oc43/KF572824/1034A_08/2008
oc43/KF572826/1081A_08/2008
oc43/KF572828/1157A_08/2008
oc43/KF572830/1216A_08/2008
oc43/KF572834/1357A_09/2009
oc43/KF572837/1593A_09/2009
oc43/KF572871/9138_09/2009
oc43/KF572872/978A_08/2008
oc43/KF572836/1591A_0

In [ ]:
def divergence_by_clade(cov, gene, window, clade, min_seqs, year_max=None, year_min=None):
    #Find fraction of sites that differ from root and average among all viruses at each time point
    input_file_root = '../'+str(cov)+'/auspice/seasonal_corona_'+str(cov)+'_'+str(gene)+'_root-sequence.json'
    input_file_alignment = '../'+str(cov)+'/results/aligned_'+str(cov)+'_'+str(gene)+'.fasta'
    metafile = '../'+str(cov)+'/results/metadata_'+str(cov)+'_'+str(gene)+'.tsv'
    
    #Subset data based on time windows
    meta = pd.read_csv(metafile, sep = '\t')
    meta.drop(meta[meta['date']=='?'].index, inplace=True)
    meta.dropna(subset=['date'], inplace=True)
    meta['year'] = meta['date'].str[:4].astype('int')
    
    if year_max:
        meta.drop(meta[meta['year']>year_max].index, inplace=True)
    if year_min:
        meta.drop(meta[meta['year']<year_min].index, inplace=True)
    
    date_range = meta['year'].max() - meta['year'].min()
    
    
    if clade!= None:
        clade_df = separate_clades(cov, gene)
        meta = meta.merge(clade_df, on='strain')
        meta.drop(meta[meta['clade']!=clade].index, inplace=True)

    #Group viruses by time windows
    virus_time_subset = {}
    if window == 'all':
        years = str(meta['year'].min()) + '-' + str(meta['year'].max())
        virus_time_subset[years] = meta['strain'].tolist()
    else:
        date_window_start = meta['year'].min()
        date_window_end = meta['year'].min() + window
        while date_window_end <= meta['year'].max():
            years = str(date_window_start) + '-' + str(date_window_end)
            strains = meta[(meta['year']>=date_window_start) & (meta['year']<date_window_end)]['strain'].tolist()
            virus_time_subset[years] = strains
            #sliding window
            date_window_end += 1
            date_window_start += 1     

    #initiate lists to record all time windows
    year_windows = []
    seqs_in_window = []
    total_subs_fractions = []
    syn_fractions = []
    nonsyn_fractions = []
    total_subs_fractions_avg = []
    syn_fractions_avg = []
    nonsyn_fractions_avg = []
    
    #track all mutations that occur in tree
    nt_muts = []
    
    #strains at first time point(to make consensus from)
    first_window = True
    first_window_strains = []
    first_window_sequences = []
    root_seq = ''
    root_aa_seq = ''
    
    for years, subset_viruses in virus_time_subset.items():
        #don't use windows with fewer than min_seqs 
        if len(subset_viruses) >= min_seqs:
            year_windows.append(years)
            seqs_in_window.append(len(subset_viruses))
    
            
            #make consensus sequence at first time point
            if first_window == True:
                first_window_strains+=subset_viruses
                with open(input_file_alignment, "r") as aligned_handle:
                    for virus in SeqIO.parse(aligned_handle, "fasta"):
                        if virus.id in first_window_strains:
                            first_window_sequences.append(virus)
                            
                first_window_alignment = MultipleSeqAlignment(first_window_sequences)
                root_seq = AlignInfo.SummaryInfo(first_window_alignment).dumb_consensus(ambiguous ='N')
                root_aa_seq = root_seq.translate()
                
                first_window = False
            
        
        
            
            fraction_subs = []
            fraction_syn = []
            fraction_nonsyn = []
            
            #calculate divergence at all subsequent time points
            with open(input_file_alignment, "r") as aligned_handle:
                for virus in SeqIO.parse(aligned_handle, "fasta"):         
#                     Only viruses in time window
                    if virus.id in subset_viruses:    
                        
                        #check
                        if len(virus.seq) != len(root_seq):
                            print(virus)
                        elif len(virus.seq) == len(root_seq):
                            count_total_unambiguous = 0
                            count_subs = 0
                            count_syn_subs = 0
                            count_nonsyn_subs = 0
                            for pos in range(len(root_seq)):
                                root_nt = str(root_seq[pos])
                                virus_nt = str(virus.seq[pos])
                                #skip ambiguous sites
                                if virus_nt != 'N':
                                    if root_nt != 'N':
                                        count_total_unambiguous+=1
                                        if virus_nt != root_nt:
                                            count_subs+=1
                                            #determine syn or nonsyn
                                            codon = math.floor(pos/3)
                                            codon_pos = pos-(codon*3)
                                            if codon_pos == 0:
                                                codon_nt = virus.seq[pos:(pos+3)]
                                            elif codon_pos == 1:
                                                codon_nt = virus.seq[(pos-1):(pos+2)]
                                            elif codon_pos == 2:
                                                codon_nt = virus.seq[(pos-2):(pos+1)]
                                            codon_aa = codon_nt.translate()
                                            root_aa = root_aa_seq[codon]
                                            #skip ambiguous
                                            if root_aa != 'X':
                                                if codon_aa != root_aa:
                                                    count_nonsyn_subs+=1
                                                    nt_mut = str(root_nt)+str(pos)+str(virus_nt)+'nonsyn'
                                                elif codon_aa == root_aa:
                                                    count_syn_subs+=1
                                                    nt_mut = str(root_nt)+str(pos)+str(virus_nt)+'syn'
                                                if nt_mut not in nt_muts:
                                                    nt_muts.append(nt_mut)
                            fraction_sub_virus = count_subs/count_total_unambiguous
                            fraction_subs.append(fraction_sub_virus)
                            fraction_syn_virus = count_syn_subs/count_total_unambiguous
                            fraction_syn.append(fraction_syn_virus)
                            fraction_nonsyn_virus = count_nonsyn_subs/count_total_unambiguous
                            fraction_nonsyn.append(fraction_nonsyn_virus)



            fraction_subs_window = sum(fraction_subs)/len(fraction_subs)
            fraction_syn_window = sum(fraction_syn)/len(fraction_syn)
            fraction_nonsyn_window = sum(fraction_nonsyn)/len(fraction_nonsyn)
            
            total_subs_fractions.append(fraction_subs)
            syn_fractions.append(fraction_syn)
            nonsyn_fractions.append(fraction_nonsyn)
            
            total_subs_fractions_avg.append(fraction_subs_window)
            syn_fractions_avg.append(fraction_syn_window)
            nonsyn_fractions_avg.append(fraction_nonsyn_window)
            
    return year_windows, seqs_in_window, total_subs_fractions, syn_fractions, nonsyn_fractions, total_subs_fractions_avg, syn_fractions_avg, nonsyn_fractions_avg, nt_muts
